In [1]:
import cx_Oracle as cx
import pandas as pd

from datetime import date

import cx_Oracle as cx
from scipy.spatial import cKDTree

import numpy as np
import geopandas as gpd
import math as math
from shapely.geometry import Point, Polygon


import requests
import pandas as pd
import time
from datetime import date,datetime
from io import StringIO
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore')

In [2]:
#export NRCELL table
def get_param_structure(obj,list_netact):
    liste_df=[]
    key_netact='ZmhhdHRhYjpBemVydHkyMSE='
    dict_url={'1':"http://na1waslb.nct8.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          '2':"http://na2waslb.nct15.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          '3':"http://na3waslb.nct16.mob.proxad.net/netact/cm/open-api/persistency/v1/query",
          '6':"http://na6waslb.nct6.mob.proxad.net/netact/cm/open-api/persistency/v1/query"}

    dict_url_2={'1':"http://na1waslb.nct8.mob.proxad.net",
            '2':"http://na2waslb.nct15.mob.proxad.net",
            '3':"http://na3waslb.nct16.mob.proxad.net",
            '6':"http://na6waslb.nct6.mob.proxad.net"}
    
    def parse_structure(input_1,input_2, moId):
            Dict1 = eval(input_1)
            dict_1={}
            for key_1, value_1 in Dict1.items():
                if isinstance(value_1, list):
                    i=1
                    for element in value_1: 
                        if isinstance(element, dict):
                            for key2,value_2 in element.items():
                                dict_1[f'{key_1}:Structure{i}:{key2}']=value_2
                        else:
                            dict_1[f'{key_1}:Value{i}']=element
                        i+=1
                else:
                    dict_1[f'{key_1}']=value_1
            df_temp=pd.DataFrame.from_dict(dict_1,orient='index')
            df_temp.reset_index(inplace =True)
            df_temp.columns=['parameter','valeur']
            df_temp['dn']=moId
            Dict_2=eval(input_2)
            df_temp['version']=Dict_2['version']
            df_temp['mo']=Dict_2['id']
            return df_temp
        
    for netact in list_netact:
        
        url=dict_url_2[netact]
        headers = {
            'Authorization': f'Basic {key_netact}',
            'Content-Type': 'application/json'
            }

        query = "/netact/cm/open-api/persistency/v1/query"
        queryMOLites = "/netact/cm/open-api/persistency/v1/queryMOLites"
        managed_obj = "/netact/cm/open-api/persistency/v1/getManagedObjects"

        

        requete_data = "{\"confId\": 1,\"moPath\": \"//"+obj+"[dn() like \'%\']\"}"

        response1 = requests.request("POST", url + queryMOLites, headers=headers, data=requete_data)
        df1=pd.DataFrame(response1.json()['result'], dtype='str')
        lst1=[f"\"{x}\"" for x in df1.moId.unique().tolist()]
        lst1=','.join(lst1)
        requete_dn = "{\"confId\": 1,\"moIds\": " +"["+ lst1 +"]"+ "}"
        response2 = requests.request("POST", url + managed_obj, headers=headers, data=requete_dn)
        df=pd.DataFrame(response2.json()['managedObjects'], dtype='str')

        df['parameters']=df.apply(lambda row: parse_structure(row.parameters,row.moClass,row.moId), axis=1)
        df=pd.concat(df.parameters.values.tolist())
        df=df[['dn','mo','version','parameter','valeur']]
        df.sort_values(['dn','parameter'], ascending=[False, True], inplace=True)
        df=df.reset_index(drop=True)
        df['na_id']=netact

        liste_df.append(df)

    df=pd.concat(liste_df,sort=False)
    df['valeur']=df['valeur'].astype(str).str.replace('true','1').str.replace('false','0').str.replace(' ','')
    df['parameter']=df['parameter'].astype(str).str.replace(' ','')
    return df

In [4]:

import time

start = time.time()

list_netact=['6']
NRCELL=get_param_structure('com.nokia.srbts.nrbts:NRCELL',list_netact)

end = time.time()
print(end - start)

170.9254801273346


In [5]:
NRCELL.head()

,dn,mo,version,parameter,valeur,na_id
0,PLMN-PLMN/MRBTS-9677/NRBTS-9677/NRCELL-7003,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,a1MeashoEnabled,0,6
1,PLMN-PLMN/MRBTS-9677/NRBTS-9677/NRCELL-7003,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,a1MeashoEnabledSA,0,6
2,PLMN-PLMN/MRBTS-9677/NRBTS-9677/NRCELL-7003,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,a1MeashoPrefNRLayerEnabled,1,6
3,PLMN-PLMN/MRBTS-9677/NRBTS-9677/NRCELL-7003,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,a1MeashoPrefNRLayerEnabledSA,0,6
4,PLMN-PLMN/MRBTS-9677/NRBTS-9677/NRCELL-7003,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,a1MeashoPrefNRLayerSsbRsrp:Structure1:a1Hyster...,0,6


In [6]:
df_nrcell=NRCELL.pivot(index=['dn','mo','version'], columns="parameter", values="valeur").reset_index()

In [7]:
df_tilt=df_nrcell[['cellName','dn','mo','version','beamSet:Structure1:tiltOffset']]

In [8]:
df_nrcell=df_nrcell[(df_nrcell["freqBandIndicatorNR"]=="78") & (df_nrcell["administrativeState"]=="2")]


In [9]:
nrcell_exp=df_nrcell[['dn','mo','version','cellName','administrativeState','actInfTddFrameMuting','infTddFrameStructure:Structure1:infFrameStructureType','infTddFrameStructure:Structure1:infGuardPeriodLength','infTddFrameStructure:Structure1:infScs','infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement',\
                     'infTddFrameStructure:Structure1:infUlDlDataSlotRatio','beamSet:Structure1:tiltOffset']]

In [10]:
nrcell_dsb=nrcell_exp[nrcell_exp["actInfTddFrameMuting"]=="1"]

In [11]:
nrcell_dsb.shape

(429, 12)

In [12]:
conora = cx.connect('fhattab'+'/'+'atoll'+'@//192.168.105.23:1521/orclpdb',encoding='iso-8859-1')
cursor = conora.cursor()

df_atoll = pd.read_sql("select * from SITES_FILTERED where (FIELDCHAR2 ='INT' or FIELDCHAR2 ='SRI') and\
(SITES_FILTERED.TYPE !='ZBFM' and SITES_FILTERED.TYPE !='FH ONLY'and SITES_FILTERED.TYPE !='ZBAU')",conora)
df2=pd.read_csv('D:\liste_sites_atoll.csv',encoding="ISO-8859-1",delimiter=";")

for i,j in zip(df_atoll,df2):
    df_atoll.rename(columns={i:j},inplace=True) 
usecols=["Nom","Longitude","Latitude","ETAT SITE - Config"]
df_atoll=df_atoll[usecols]

In [13]:
df_site_5g = pd.read_sql("select SITE_NAME,ATOLL_5GMRAT.xgtransmitters.TX_ID,FBAND,ANTENNA_NAME,AZIMUT,LONGITUDE, LATITUDE,\
                       HEIGHT, TYPE,FIELDCHAR2\
                       from ATOLL_5GMRAT.xgtransmitters,ATOLL_5GMRAT.sites\
                       where SITE_NAME= NAME AND (FBAND ='n78')\
                       And (FIELDCHAR2 ='INT'  )\
                       And (ATOLL_5GMRAT.sites.MODIF ='ON' OR ATOLL_5GMRAT.sites.MODIF ='ON REMPLA')\
                       ", conora)


In [14]:
df_site_5g.head()

,SITE_NAME,TX_ID,FBAND,ANTENNA_NAME,AZIMUT,LONGITUDE,LATITUDE,HEIGHT,TYPE,FIELDCHAR2
0,31516_002,31516_002-3501,n78,AEQE-120degAzOp-p00-ap06-SSB#4,20,1.400245,43.742561,20.0,MACRO,INT
1,31516_002,31516_002-3502,n78,AEQE-120degAzOp-p00-ap06-SSB#4,190,1.400245,43.742561,22.0,MACRO,INT
2,57642_007,57642_007-3501,n78,AEQE-120degAzOp-p00-ap06-SSB#4,60,6.128303,49.105841,24.6,MACRO,INT
3,57642_007,57642_007-3502,n78,AEQE-120degAzOp-p00-ap06-SSB#4,240,6.128303,49.105841,24.6,MACRO,INT
4,57642_007,57642_007-3503,n78,AEQE-120degAzOp-p00-ap06-SSB#4,330,6.128303,49.105841,24.6,MACRO,INT


In [15]:
df_points=pd.merge(nrcell_exp,df_site_5g[["SITE_NAME","TX_ID","AZIMUT","LONGITUDE","LATITUDE"]],left_on="cellName",right_on='TX_ID',how="left")

In [16]:
df_points.head()

,dn,mo,version,cellName,administrativeState,actInfTddFrameMuting,infTddFrameStructure:Structure1:infFrameStructureType,infTddFrameStructure:Structure1:infGuardPeriodLength,infTddFrameStructure:Structure1:infScs,infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement,infTddFrameStructure:Structure1:infUlDlDataSlotRatio,beamSet:Structure1:tiltOffset,SITE_NAME,TX_ID,AZIMUT,LONGITUDE,LATITUDE
0,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3501,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3501,0.0,-0.776111,48.075278
1,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3502,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3502,120.0,-0.776111,48.075278
2,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3503,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3503,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3503,240.0,-0.776111,48.075278
3,PLMN-PLMN/MRBTS-1008/NRBTS-1008/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,59155_008-3501,2,0,NaN,NaN,NaN,NaN,NaN,NaN,59155_008,59155_008-3501,60.0,2.406890,51.030000
4,PLMN-PLMN/MRBTS-1008/NRBTS-1008/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,59155_008-3502,2,0,NaN,NaN,NaN,NaN,NaN,NaN,59155_008,59155_008-3502,230.0,2.406890,51.030000


In [17]:
df_points.rename(columns={"LONGITUDE":"longitude","LATITUDE":"latitude"},inplace='True')

In [18]:
#import france shape
borders = gpd.read_file('france_border_abd_wgs_mod4.geojson')

In [19]:
#convert shape to infinity of points
points = []
for polygon in borders.iloc[0]['geometry']:
    points.extend(polygon.exterior.coords[:-1])


In [20]:
#create geodataframe from the points of borders
gdf_bor=pd.DataFrame(points,columns =['LONGITUDE', 'LATITUDE']) 
print(gdf_bor)
gdf_bor=gpd.GeoDataFrame(gdf_bor, geometry=gpd.points_from_xy(gdf_bor.LONGITUDE,gdf_bor.LATITUDE),crs="EPSG:4326")

        LONGITUDE  LATITUDE
0        45.02737 -12.65786
1        45.02702 -12.65815
2        45.02693 -12.65794
3        45.02691 -12.65754
4        45.02701 -12.65740
...           ...       ...
150094   45.17350 -12.71845
150095   45.17327 -12.71881
150096   45.17314 -12.71866
150097   45.17342 -12.71834
150098   45.17372 -12.71816

[150099 rows x 2 columns]


In [21]:
df_points.head()

,dn,mo,version,cellName,administrativeState,actInfTddFrameMuting,infTddFrameStructure:Structure1:infFrameStructureType,infTddFrameStructure:Structure1:infGuardPeriodLength,infTddFrameStructure:Structure1:infScs,infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement,infTddFrameStructure:Structure1:infUlDlDataSlotRatio,beamSet:Structure1:tiltOffset,SITE_NAME,TX_ID,AZIMUT,longitude,latitude
0,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3501,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3501,0.0,-0.776111,48.075278
1,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3502,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3502,120.0,-0.776111,48.075278
2,PLMN-PLMN/MRBTS-1005/NRBTS-1005/NRCELL-3503,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,53130_006-3503,2,0,NaN,NaN,NaN,NaN,NaN,NaN,53130_006,53130_006-3503,240.0,-0.776111,48.075278
3,PLMN-PLMN/MRBTS-1008/NRBTS-1008/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,59155_008-3501,2,0,NaN,NaN,NaN,NaN,NaN,NaN,59155_008,59155_008-3501,60.0,2.406890,51.030000
4,PLMN-PLMN/MRBTS-1008/NRBTS-1008/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,59155_008-3502,2,0,NaN,NaN,NaN,NaN,NaN,NaN,59155_008,59155_008-3502,230.0,2.406890,51.030000


In [22]:
gpd_site = gpd.GeoDataFrame(df_points, geometry=gpd.points_from_xy(df_points.longitude,df_points.latitude),crs={'init' :'epsg:4326'}) 
gpd_site.dtypes

dn                                                                object
mo                                                                object
version                                                           object
cellName                                                          object
administrativeState                                               object
actInfTddFrameMuting                                              object
infTddFrameStructure:Structure1:infFrameStructureType             object
infTddFrameStructure:Structure1:infGuardPeriodLength              object
infTddFrameStructure:Structure1:infScs                            object
infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement      object
infTddFrameStructure:Structure1:infUlDlDataSlotRatio              object
beamSet:Structure1:tiltOffset                                     object
SITE_NAME                                                         object
TX_ID                                              

In [23]:
gpd1=gpd_site
gpd2=gdf_bor

In [24]:
gpd1=gpd1[(gpd1["longitude"].notnull()) & (gpd1["latitude"].notnull())]
gpd1=gpd1[gpd1["SITE_NAME"].str.startswith(("68","57","54","55","67"))]

nA = np.array(list(gpd1.geometry.apply(lambda x: (x.x, x.y))))
nB = np.array(list(gpd2.geometry.apply(lambda x: (x.x, x.y))))


btree = cKDTree(nB)
dist, idx = btree.query(nA, k=2)

df_b=pd.DataFrame()

for i in range (0,len(idx.tolist())):
    print(idx.tolist()[i])
   
    df_b=df_b.append(pd.concat([pd.concat([gpd1.iloc[i].to_frame().T]*2).reset_index(),gpd2.iloc[idx.tolist()[i],:].drop(columns="geometry").reset_index(drop=True),pd.Series(dist[0],name='dist')],axis=1))
 

[124901, 124900]
[124901, 124900]
[124901, 124900]
[124251, 124288]
[124251, 124288]
[124251, 124288]
[125036, 125035]
[125036, 125035]
[125341, 125340]
[125341, 125340]
[125341, 125340]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[128323, 128322]
[128323, 128322]
[128323, 128322]
[128626, 128625]
[128626, 128625]
[128626, 128625]
[125407, 125408]
[125407, 125408]
[125407, 125408]
[128768, 128767]
[128768, 128767]
[128768, 128767]
[123724, 123725]
[123724, 123725]
[123724, 123725]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[129754, 129753]
[129754, 129753]
[129754, 129753]
[128646, 128647]
[128646, 128647]
[128646, 128647]
[124612, 124629]
[124612, 124629]
[124612, 124629]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124246, 124247]
[124246, 124247]
[124246, 124247]
[124044, 124040]
[124044, 124040]
[124044, 124040]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124288, 124287]
[124288, 124287]
[124288, 12428

[128809, 128810]
[126438, 126437]
[126438, 126437]
[126438, 126437]
[128770, 128769]
[128770, 128769]
[128770, 128769]
[124221, 124220]
[124221, 124220]
[124221, 124220]
[128394, 128393]
[128394, 128393]
[128394, 128393]
[124251, 124250]
[124251, 124250]
[128512, 128513]
[128512, 128513]
[128512, 128513]
[124977, 124978]
[124977, 124978]
[124977, 124978]
[124909, 124908]
[124909, 124908]
[124909, 124908]
[124251, 124246]
[124251, 124246]
[124251, 124246]
[128528, 128527]
[128528, 128527]
[128528, 128527]
[128771, 128770]
[128771, 128770]
[128771, 128770]
[128771, 128770]
[128809, 128808]
[128809, 128808]
[128809, 128808]
[128373, 128372]
[128373, 128372]
[128373, 128372]
[124288, 124287]
[124288, 124287]
[124288, 124287]
[128809, 128810]
[128809, 128810]
[128809, 128810]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[128512, 128511]
[128512, 128511]
[128809, 128810]
[128809, 128810]
[128809, 128810]
[124901, 124900]
[124901, 124900]
[124901, 124900]
[124288, 124287]
[124288, 12428

In [25]:
def haversine_vectorize(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2=lon1.astype("float"),lat1.astype("float"),lon2.astype("float"),lat2.astype("float")
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    newlon = lon2 - lon1
    newlat = lat2 - lat1
        
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [26]:
df_b["distance_Km"]=haversine_vectorize(df_b["longitude"],df_b["latitude"],df_b["LONGITUDE"],df_b["LATITUDE"])

In [27]:
df_b["dist_min"]=df_b.groupby(['cellName'])["distance_Km"].transform('min')

In [28]:
df_act=df_b[(df_b["actInfTddFrameMuting"]=="1") & (df_b["distance_Km"]==df_b["dist_min"])]

In [29]:
df_20km=df_b[(df_b["distance_Km"]<=40 )& (df_b["distance_Km"]==df_b["dist_min"])]
df_20km.shape

(568, 24)

In [30]:
df_20km.head()

,index,dn,mo,version,cellName,administrativeState,actInfTddFrameMuting,infTddFrameStructure:Structure1:infFrameStructureType,infTddFrameStructure:Structure1:infGuardPeriodLength,infTddFrameStructure:Structure1:infScs,...,TX_ID,AZIMUT,longitude,latitude,geometry,LONGITUDE,LATITUDE,dist,distance_Km,dist_min
1,53,PLMN-PLMN/MRBTS-11475/NRBTS-11475/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,57207_005-3501,2,0,NaN,NaN,NaN,...,57207_005-3501,90,6.87145,49.124,POINT (6.871453 49.123966),6.847532,49.157117,0.714911,4.073792,4.073792
1,54,PLMN-PLMN/MRBTS-11475/NRBTS-11475/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,57207_005-3502,2,0,NaN,NaN,NaN,...,57207_005-3502,200,6.87145,49.124,POINT (6.871453 49.123966),6.847532,49.157117,0.714911,4.073792,4.073792
0,64,PLMN-PLMN/MRBTS-11992/NRBTS-11992/NRCELL-3501,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,57227_006-3501,2,1,2,2,30,...,57227_006-3501,80,6.8842,49.1921,POINT (6.884202 49.192127),6.888937,49.210133,0.714693,2.030242,2.030242
0,65,PLMN-PLMN/MRBTS-11992/NRBTS-11992/NRCELL-3502,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,57227_006-3502,2,1,2,2,30,...,57227_006-3502,210,6.8842,49.1921,POINT (6.884202 49.192127),6.888937,49.210133,0.714693,2.030242,2.030242
0,66,PLMN-PLMN/MRBTS-11992/NRBTS-11992/NRCELL-3503,com.nokia.srbts.nrbts:NRCELL,NRBTSCL23R3_2309_110,57227_006-3503,2,1,2,2,30,...,57227_006-3503,320,6.8842,49.1921,POINT (6.884202 49.192127),6.888937,49.210133,0.714693,2.030242,2.030242


In [31]:
#df_cell=pd.merge(df_site_5g[[SITE_NAME','TX_ID','AZIMUT']],df_5km,how='inner',left_on="SITE_NAME",right_on="Nom")

In [32]:
df_sect=df_20km[(df_20km.AZIMUT<200) | (df_20km.AZIMUT>340)]

In [33]:
#secteur not activated

df_notact=df_sect[(df_sect["actInfTddFrameMuting"]=="0") & (df_sect['infTddFrameStructure:Structure1:infGuardPeriodLength']!="2")\
        & (df_sect["infTddFrameStructure:Structure1:infFrameStructureType"]!="2")\
        & (df_sect["infTddFrameStructure:Structure1:infScs"]!="30")\
        & (df_sect["infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement"]!="1") &\
        (df_sect["infTddFrameStructure:Structure1:infUlDlDataSlotRatio"]!="5")]

In [34]:
#secteur not activated

df_inco=df_sect[(df_sect["actInfTddFrameMuting"]=="0") ^(df_sect['infTddFrameStructure:Structure1:infGuardPeriodLength']!="2")\
        ^(df_sect["infTddFrameStructure:Structure1:infFrameStructureType"]!="2")\
        ^ (df_sect["infTddFrameStructure:Structure1:infScs"]!="30")\
        ^ (df_sect["infTddFrameStructure:Structure1:infTddFrameMutingBiAgreement"]!="1") \
        ^(df_sect["infTddFrameStructure:Structure1:infUlDlDataSlotRatio"]!="5")]

In [35]:
with pd.ExcelWriter('D:\DSB-Activate_part-2'+datetime.today().strftime('%F')+'.xlsx') as writer:  
    df_20km.to_excel(writer, sheet_name='BTS-20KM-')
    df_sect.to_excel(writer, sheet_name="secteurs-Km")
    df_notact.to_excel(writer, sheet_name='need-activation')
    df_inco.to_excel(writer, sheet_name="inco")
    df_act.to_excel(writer, sheet_name="all_activated")

In [36]:
nrcell_dsb.to_csv("dsb_act_05_09.csv")

In [37]:
nrcell_exp[nrcell_exp["cellName"].str.startswith(("57","67","01","25","39","68","73","74","90"))].to_csv("dsb_act_05_09_all_cell.csv")